In [ ]:
import time
from os import path
import requests
import json
import copy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats
from sklearn.linear_model import LinearRegression

In [ ]:

#localityNos = [11756, 11754, 11667, 11652, 10080, 14435, 13965, 11738, 10086, 39577, 10309, 13704]

        

In [ ]:
organizationsNameToNo = {}

with open('organizations.json', encoding='utf-8') as json_file:
    data = json.load(json_file)
    
    for organization in data:
        organizationsNameToNo[organization['name']] = organization['orgNo']

In [ ]:
organizationsNameToNo

In [ ]:
def getLocalityNamesAndNos(organizationName):
    if organizationName not in organizationsNameToNo:
        raise Exception('Bad organization name')

    organizationNo = organizationsNameToNo[organizationName]
    
    fileName = './organizations/%i.json' % (organizationNo, )
    
    if path.exists(fileName):
        with open(fileName, encoding='utf-8') as json_file:
            response = json.load(json_file)
    else:
        time.sleep(1)
        
        url = 'https://www.barentswatch.no/bwapi/v1/geodata/fishhealth/locality/2020/18?organization=%i&tagFilteredLocalities=true' % (organizationNo, )

        response = json.loads(requests.get(url).text)
    
        with open(fileName, 'w') as f:
            f.write(json.dumps(response, indent=4))
            f.close()
    
    localitiesNameToNo = {}
    localitiesNameToLatitude = {}
    
    for locality in response['localities']:
        localitiesNameToNo[locality['name']] = locality['localityNo']
        localitiesNameToLatitude[locality['name']] = locality['lat']

    localityNames = [locality['name'] for locality in response['localities'] if locality['inFilteredSelection']]
    
    localityNames.sort()
    
    localityNamesAndNos = [ (localityName, localitiesNameToNo[localityName], localitiesNameToLatitude[localityName]) for localityName in localityNames ]

    return localityNamesAndNos

organizationName = 'BLOM FISKEOPPDRETT AS'
localityNames = getLocalityNamesAndNos(organizationName)

print(localityNames)

In [ ]:
df = pd.read_csv('2012_2020_18-fixed-3.csv', index_col=0).sort_index()

In [ ]:
df

In [ ]:
vikane = df.loc[df['localityNo'] == 13704]   

In [ ]:
vikane

In [ ]:
vikane = 13704

#localityNos = [ vikane ]
#[ 13704, 11756, 12982 ]

#localityNos = np.unique(df['localityNo'].to_numpy())[1:100]

localityNos = [11756, 11754, 11667, 11652, 10080, 14435, 13965, 11738, 10086, 39577, 10309, 13704]

# Calculate growing periods
def getPeriods(localityNamesAndNos, startYear, startWeek, endYear, endWeek):
    startYearWeek = startYear * 52 + startWeek
    endYearWeek = endYear * 52 + endWeek
        
    periods = []

    for (localityName, localityNo, localityLatitude) in localityNamesAndNos:
        vikane = df.loc[df['localityNo'] == localityNo]    

        lastYearWeek = { 'year': None, 'week': None, 'yearweek': None }
        currentPeriod = { 'localityNo': localityNo, 'startYear': None, 'startWeek': None, 'endYear': None, 'endWeek': None }

        counter = 0

        for index, row in vikane.iterrows():
            counter = counter + 1
            currentYearWeek = row['year'] * 52 + row['week']
            
            if currentYearWeek < startYearWeek or currentYearWeek > endYearWeek:
                continue

            if not lastYearWeek['yearweek']:
                currentPeriod['startYear'] = row['year']
                currentPeriod['startWeek'] = row['week']
            elif lastYearWeek['yearweek'] and not (lastYearWeek['yearweek'] == currentYearWeek - 1 or lastYearWeek['yearweek'] == currentYearWeek - 2):
                currentPeriod['endYear'] = lastYearWeek['year']
                currentPeriod['endWeek'] = lastYearWeek['week']

                periods.append(copy.copy(currentPeriod))

                currentPeriod['startYear'] = row['year']
                currentPeriod['startWeek'] = row['week']
                currentPeriod['endYear'] = None
                currentPeriod['endWeek'] = None
            elif counter == len(vikane):
                currentPeriod['endYear'] = row['year']
                currentPeriod['endWeek'] = row['week']

                periods.append(copy.copy(currentPeriod))

            lastYearWeek['year'] = row['year']
            lastYearWeek['week'] = row['week']
            lastYearWeek['yearweek'] = currentYearWeek

    return periods
#for period in periods:
#    print(period)

In [ ]:
def getTreatmentInfo(periods):
    allTreatmentInfo = []

    #periods = [ {'localityNo': 10090, 'startYear': 2016, 'startWeek': 1, 'endYear': 2016, 'endWeek': 35} ]

    for period in periods:
        localityNo = period['localityNo']
        startYearWeek = period['startYear'] * 52 + period['startWeek']
        endYearWeek = period['endYear'] * 52 + period['endWeek']

        treatmentInfo = []

        lastTreatmentYearWeek = None
        lastRow = None
        row = None
        lastLevelsBeforeTreatment = None
        lastLevelsAfterTreatment = None

        vikane = df.loc[df['localityNo'] == localityNo]

        counter = 0

        for index, nextRow in vikane.iterrows():
            counter = counter + 1

            if lastRow is not None:
                currentYearWeek = row['year'] * 52 + row['week']

                if currentYearWeek < startYearWeek or currentYearWeek > endYearWeek:
                    lastRow = row
                    row = nextRow

                    continue

                if row['hasMechanicalRemoval'] or row['hasBathTreatment'] or (counter == len(vikane) and (nextRow['hasMechanicalRemoval'] or nextRow['hasBathTreatment'])):
                    beforeRow = lastRow if lastRow['avgAdultFemaleLice'] > row['avgAdultFemaleLice'] else row
                    afterRow = nextRow if lastRow['avgAdultFemaleLice'] < row['avgAdultFemaleLice'] else row

                    currentLevelsBeforeTreatment = { 'avgAdultFemaleLice': beforeRow['avgAdultFemaleLice'], 'avgMobileLice': beforeRow['avgMobileLice'], 'avgStationaryLice': beforeRow['avgStationaryLice'] }
                    currentLevelsAfterTreatment = { 'avgAdultFemaleLice': afterRow['avgAdultFemaleLice'], 'avgMobileLice': afterRow['avgMobileLice'], 'avgStationaryLice': afterRow['avgStationaryLice'] }

                    treatmentYearWeek = afterRow['year'] * 52 + afterRow['week']

                    if lastTreatmentYearWeek:    
                        weeks = treatmentYearWeek - lastTreatmentYearWeek
                        treatmentInfo.append(copy.copy({ 'hasMechanicalRemoval': row['hasMechanicalRemoval'] or nextRow['hasMechanicalRemoval'], 'hasBathTreatment': row['hasBathTreatment'] or nextRow['hasBathTreatment'], 'lastLevelsBeforeTreatment': lastLevelsBeforeTreatment, 'lastLevelsAfterTreatment': lastLevelsAfterTreatment, 'currentLevelsBeforeTreatment': currentLevelsBeforeTreatment, 'currentLevelsAfterTreatment': currentLevelsAfterTreatment, 'weeks': weeks }))

                    lastTreatmentYearWeek = treatmentYearWeek

                    lastLevelsBeforeTreatment = currentLevelsBeforeTreatment
                    lastLevelsAfterTreatment = currentLevelsAfterTreatment


                lastRow = row
                row = nextRow
            elif row is not None:
                lastRow = row 
                row = nextRow
            else:
                row = nextRow

        allTreatmentInfo = allTreatmentInfo + treatmentInfo

    return allTreatmentInfo
#print(len(allTreatmentInfo))

In [ ]:
#allTreatmentInfo[0]

In [ ]:
def getTreatmentMetrics(currentTreatmentInfo):
    beforeTreatmentAdultFemale = np.array([ info['lastLevelsBeforeTreatment']['avgAdultFemaleLice'] for info in currentTreatmentInfo ])
    beforeTreatmentMobile = np.array([ info['lastLevelsBeforeTreatment']['avgMobileLice'] for info in currentTreatmentInfo ])
    beforeTreatmentStationary = np.array([ info['lastLevelsBeforeTreatment']['avgStationaryLice'] for info in currentTreatmentInfo ])

    afterTreatmentAdultFemale = np.array([ info['lastLevelsAfterTreatment']['avgAdultFemaleLice'] for info in currentTreatmentInfo ])
    afterTreatmentMobile = np.array([ info['lastLevelsAfterTreatment']['avgMobileLice'] for info in currentTreatmentInfo ])
    afterTreatmentStationary = np.array([ info['lastLevelsAfterTreatment']['avgStationaryLice'] for info in currentTreatmentInfo ])

    allTreatmentWeeks = np.array([ info['weeks'] for info in currentTreatmentInfo ])
    allMechanicalTreatments = np.array([ info['hasMechanicalRemoval'] for info in currentTreatmentInfo ])
    allBathTreatments = np.array([ info['hasBathTreatment'] for info in currentTreatmentInfo ])

    treatmentEffectAdultFemale = beforeTreatmentAdultFemale - afterTreatmentAdultFemale
    treatmentEffectMobile = beforeTreatmentMobile - afterTreatmentMobile
    treatmentEffectStationary = beforeTreatmentStationary - afterTreatmentStationary

    colors = ['r' if t else 'g' for t in allMechanicalTreatments]
    
    return {
        'treatmentInfo': currentTreatmentInfo,
        
        'beforeTreatmentAdultFemale': beforeTreatmentAdultFemale,
        'beforeTreatmentMobile': beforeTreatmentMobile,
        'beforeTreatmentStationary': beforeTreatmentStationary,
        
        'afterTreatmentAdultFemale': afterTreatmentAdultFemale,
        'afterTreatmentMobile': afterTreatmentMobile,
        'afterTreatmentStationary': afterTreatmentStationary,
        
        'allTreatmentWeeks': allTreatmentWeeks,
        'allMechanicalTreatments': allMechanicalTreatments,
        'allBathTreatments': allBathTreatments,
        
        'treatmentEffectAdultFemale': treatmentEffectAdultFemale,
        'treatmentEffectMobile': treatmentEffectMobile,
        'treatmentEffectStationary': treatmentEffectStationary,
        
        'colors': colors
    }

In [ ]:


#plt.scatter(allMetrics['treatmentEffectAdultFemale'], allMetrics['allTreatmentWeeks'], c=colors)

In [ ]:
def calculateGrowthWeeks(liceLevels, allTreatmentWeeks, subset):
    x = liceLevels[subset]
    Y = allTreatmentWeeks[subset]

    X = x.reshape(-1, 1)

    #plt.plot(X, Y, 'p')

    reg = LinearRegression().fit(X, Y)

    #print(reg.intercept_, reg.coef_)
    #print(reg.score(X, Y))
    
    return (reg.coef_ * -1, reg.score(X, Y))

In [ ]:
def printMetrics(metrics):
    subsetTEAdultFemale = np.where((metrics['treatmentEffectAdultFemale'] < 2) & (metrics['treatmentEffectAdultFemale'] >= 0))
    subsetTEMobile = np.where((metrics['treatmentEffectMobile'] < 7) & (metrics['treatmentEffectMobile'] >= 0))
    subsetTEStationary = np.where((metrics['treatmentEffectStationary'] < 5) & (metrics['treatmentEffectStationary'] >= 0))

    if np.sum(subsetTEAdultFemale) > 0 and np.sum(subsetTEMobile) > 0 and np.sum(subsetTEStationary) > 0:
        (treatmentEffectWeeksAdultFemale, treatmentEffectRSQAdultFemale) = calculateGrowthWeeks(metrics['treatmentEffectAdultFemale'], metrics['allTreatmentWeeks'], subsetTEAdultFemale)
        (treatmentEffectWeeksMobile, treatmentEffectRSQMobile) = calculateGrowthWeeks(metrics['treatmentEffectMobile'], metrics['allTreatmentWeeks'], subsetTEMobile)
        (treatmentEffectWeeksStationary, treatmentEffectRSQStationary) = calculateGrowthWeeks(metrics['treatmentEffectStationary'], metrics['allTreatmentWeeks'], subsetTEStationary)
    else:
        (treatmentEffectWeeksAdultFemale, treatmentEffectRSQAdultFemale) = (0, 0)
        (treatmentEffectWeeksMobile, treatmentEffectRSQMobile) = (0, 0)
        (treatmentEffectWeeksStationary, treatmentEffectRSQStationary) = (0, 0)

    subsetBTAdultFemale = np.where((metrics['beforeTreatmentAdultFemale'] < 1))
    subsetBTMobile = np.where((metrics['beforeTreatmentMobile'] < 10))
    subsetBTStationary = np.where((metrics['beforeTreatmentStationary'] < 5))

    if np.sum(subsetBTAdultFemale) > 0 and np.sum(subsetBTMobile) > 0 and np.sum(subsetBTStationary) > 0:
        (beforeTreatmentWeeksAdultFemale, beforeTreatmentRSQAdultFemale) = calculateGrowthWeeks(metrics['beforeTreatmentAdultFemale'], metrics['allTreatmentWeeks'], subsetBTAdultFemale)
        (beforeTreatmentWeeksMobile, beforeTreatmentRSQMobile) = calculateGrowthWeeks(metrics['beforeTreatmentMobile'], metrics['allTreatmentWeeks'], subsetBTMobile)
        (beforeTreatmentWeeksStationary, beforeTreatmentRSQStationary) = calculateGrowthWeeks(metrics['beforeTreatmentStationary'], metrics['allTreatmentWeeks'], subsetBTStationary)
    else:
        (beforeTreatmentWeeksAdultFemale, beforeTreatmentRSQAdultFemale) = (0, 0)
        (beforeTreatmentWeeksMobile, beforeTreatmentRSQMobile) = (0, 0)
        (beforeTreatmentWeeksStationary, beforeTreatmentRSQStationary) = (0, 0)
                                                                                              
    subsetATAdultFemale = np.where((metrics['afterTreatmentAdultFemale'] < 0.5))
    subsetATMobile = np.where((metrics['afterTreatmentMobile'] < 1))
    subsetATStationary = np.where((metrics['afterTreatmentStationary'] < 1))

    if np.sum(subsetATAdultFemale) > 0 and np.sum(subsetATMobile) > 0 and np.sum(subsetATStationary) > 0:
        (afterTreatmentWeeksAdultFemale, afterTreatmentRSQAdultFemale) = calculateGrowthWeeks(metrics['afterTreatmentAdultFemale'], metrics['allTreatmentWeeks'], subsetATAdultFemale)
        (afterTreatmentWeeksMobile, afterTreatmentRSQMobile) = calculateGrowthWeeks(metrics['afterTreatmentMobile'], metrics['allTreatmentWeeks'], subsetATMobile)
        (afterTreatmentWeeksStationary, afterTreatmentRSQStationary) = calculateGrowthWeeks(metrics['afterTreatmentStationary'], metrics['allTreatmentWeeks'], subsetATStationary)
    else:
        (afterTreatmentWeeksAdultFemale, afterTreatmentRSQAdultFemale) = (0, 0)
        (afterTreatmentWeeksMobile, afterTreatmentRSQMobile) = (0, 0)
        (afterTreatmentWeeksStationary, afterTreatmentRSQStationary) = (0, 0)

    metricsText = [
        '',
        'Weeks between treatment: %0.2f (%i days)' % (np.mean(metrics['allTreatmentWeeks']), np.mean(metrics['allTreatmentWeeks']) * 7),
        
        '',
        'Treatment Effectiveness',
        '              Before   After   Pct Removed',
        'Adult Female  %0.2f     %0.2f    %i%%' % (np.mean(metrics['beforeTreatmentAdultFemale']), np.mean(metrics['afterTreatmentAdultFemale']), (1 - np.mean(metrics['afterTreatmentAdultFemale']) / np.mean(metrics['beforeTreatmentAdultFemale'])) * 100),   
        'Mobile        %0.2f     %0.2f    %i%%' % (np.mean(metrics['beforeTreatmentMobile']), np.mean(metrics['afterTreatmentMobile']), (1 - np.mean(metrics['afterTreatmentMobile']) / np.mean(metrics['beforeTreatmentMobile'])) * 100),
        'Stationary    %0.2f     %0.2f    %i%%' % (np.mean(metrics['beforeTreatmentStationary']), np.mean(metrics['afterTreatmentStationary']), (1 - np.mean(metrics['afterTreatmentStationary']) / np.mean(metrics['beforeTreatmentStationary'])) * 100),
        
        '',
        #'Additional weeks between treatment per removed lice',
        #'AF: %0.2f (R^2 = %0.2f)' % (treatmentEffectWeeksAdultFemale, treatmentEffectRSQAdultFemale),
        #'M: %0.2f (R^2 = %0.2f)' % (treatmentEffectWeeksMobile, treatmentEffectRSQMobile),
        #'S: %0.2f (R^2 = %0.2f)' % (treatmentEffectWeeksStationary, treatmentEffectRSQStationary),
        #'Additional weeks between treatment per lice (before)',
        #'AF: %0.2f (R^2 = %0.2f)' % (beforeTreatmentWeeksAdultFemale, beforeTreatmentRSQAdultFemale),
        #'M: %0.2f (R^2 = %0.2f)' % (beforeTreatmentWeeksMobile, beforeTreatmentRSQMobile),
        #'S: %0.2f (R^2 = %0.2f)' % (beforeTreatmentWeeksStationary, beforeTreatmentRSQStationary),
        
        '',
        'Additional weeks between treatment per lice (after)',
        'Adult Female  %0.2f (R^2 = %0.2f)' % (afterTreatmentWeeksAdultFemale, afterTreatmentRSQAdultFemale),
        'Mobile        %0.2f (R^2 = %0.2f)' % (afterTreatmentWeeksMobile, afterTreatmentRSQMobile),
        'Stationary    %0.2f (R^2 = %0.2f)' % (afterTreatmentWeeksStationary, afterTreatmentRSQStationary)
    ]

    for metric in metricsText:
        print(metric)

def printReport(allTreatmentInfo, organizationName, localityNames, startYear, startWeek, endYear, endWeek):
    mechanicalTreatmentInfos = [ info for info in allTreatmentInfo if info['hasMechanicalRemoval'] ]
    bathTreatmentInfos = [ info for info in allTreatmentInfo if info['hasBathTreatment'] ]

    allMetrics = getTreatmentMetrics(allTreatmentInfo)
    mechanicalMetrics = getTreatmentMetrics(mechanicalTreatmentInfos)
    bathMetrics = getTreatmentMetrics(bathTreatmentInfos)

    print('FARM: %s' % (organizationName, ))
    print('SITES: %s' % (', '.join(localityNames)))

    print()
    print('START DATE: %i Week %i' % (startYear, startWeek))
    print('END DATE: %i Week %i' % (endYear, endWeek))
    print()

    print('----------')


    print()
    print('ALL TREATMENTS (%i)' % (len(allMetrics['treatmentInfo'], )))
    printMetrics(allMetrics)

    print()
    print('----------')
    print()
    print('MECHANICAL TREATMENTS (%i)' % (len(mechanicalMetrics['treatmentInfo'], )))
    printMetrics(mechanicalMetrics)

    print()
    print('----------')
    print()
    print('BATH TREATMENTS (%i)' % (len(bathMetrics['treatmentInfo'], )))
    printMetrics(bathMetrics)

In [ ]:
def getMobileAnalysis(organizationName, startYear, startWeek, endYear, endWeek):
    localityNamesAndNos = getLocalityNamesAndNos(organizationName)
    localityNames = [ l[0] for l in localityNamesAndNos ]

    if len(localityNames) == len(localityNamesAndNos):
        localityNames = [ 'ALL' ]

    periods = getPeriods(localityNamesAndNos, startYear, startWeek, endYear, endWeek)

    allTreatmentInfo = getTreatmentInfo(periods)

    if len(allTreatmentInfo) > 0:
        mechanicalTreatmentInfos = [ info for info in allTreatmentInfo if info['hasMechanicalRemoval'] ]
        bathTreatmentInfos = [ info for info in allTreatmentInfo if info['hasBathTreatment'] ]

        allMetrics = getTreatmentMetrics(allTreatmentInfo)
        mechanicalMetrics = getTreatmentMetrics(mechanicalTreatmentInfos)
        bathMetrics = getTreatmentMetrics(bathTreatmentInfos)
        
        mechanicalTreatmentWeeks = np.mean(mechanicalMetrics['allTreatmentWeeks'])
        mechanicalBeforeTreatmentMobile = np.mean(mechanicalMetrics['beforeTreatmentMobile'])
        
        return (mechanicalTreatmentWeeks, localityNamesAndNos[0][2], len(mechanicalTreatmentInfos))
        #return (mechanicalTreatmentWeeks, mechanicalBeforeTreatmentMobile, len(mechanicalTreatmentInfos))
    
    return (None, None, None)

print(getMobileAnalysis('NORDLAKS OPPDRETT AS', startYear, startWeek, endYear, endWeek))

In [ ]:
startYear = 2012
startWeek = 1

endYear = 2020
endWeek = 18

# 'NORDFJORD FORSØKSSTASJON AS': 996997979,
#  'NORDFJORD HAVBRUK AS': 976583531,
#  'NORDFJORD LAKS AS': 921398336,
#  'NORDFJORD LAKS HOLDING AS': 976583531,
#  'NORDFJORD OPPDRETT AS': 921398336,
#  'NORDFJORD TORSK AS': 993209678,
#  'NORDIC HALIBUT AS': 974526441,
#  'NORDLAKS OPPDRETT AS': 955750802,
#  'NORDLAKS PRODUKTER AS': 976725859,
#  'NORDLAKS SMOLT AS': 991193332,
    
#organizationName = 'BLOM FISKEOPPDRETT AS'
#organizationName = 'ARNØY LAKS AS'
#organizationName = 'NORDFJORD OPPDRETT AS'
organizationName = 'NORDLAKS OPPDRETT AS'

def getReport(organizationName, startYear, startWeek, endYear, endWeek):
    localityNamesAndNos = getLocalityNamesAndNos(organizationName)
    localityNames = [ l[0] for l in localityNamesAndNos ]

    if len(localityNames) == len(localityNamesAndNos):
        localityNames = [ 'ALL' ]

    #periods = getPeriods(localityNamesAndNos, startYear, startWeek, endYear, endWeek)

    #allTreatmentInfo = getTreatmentInfo(periods)

    if len(allTreatmentInfo) > 0:
        pass
        #printReport(allTreatmentInfo, organizationName, localityNames, startYear, startWeek, endYear, endWeek)

In [ ]:
mechanicalTreatmentWeeksHistory = {}
mechanicalBeforeTreatmentMobileHistory = {}
numMechanicalTreatmentsHistory = {}
organizationLatitudeHistory = {}

In [ ]:
startYear = 2012
startWeek = 1

endYear = 2020
endWeek = 18

duplicateOrganizationList = [
    'AQUA FARMS VARTDAL A/S',
    'AQUAGEN AS',
    'BENCHMARK GENETICS NORWAY AS',
    'BJØRØYA FISKEOPPDRETT AS',
    'BLOM SETTEFISK AS',
    'BOLSTAD FJORDBRUK AS',
    'EIDE FJORDBRUK A/S',
    'EUROPHARMA AS',
    'FJELBERG FJORDBRUK A/S',
    'FJORDLAKS AQUA AS',
    'FLOKENES FISKEFARM A/S',
    'GRATANGLAKS AS',
    'HAVBRUKSAKADEMIET AS',
    'HELGELAND HAVBRUKSSTASJON AS',
    'ILSVÅG HOLDING AS',
    'KARSTENSEN E FISKEOPPDRETT AS',
    'KOBBVÅGLAKS A/S',
    'LERØY MIDNOR AS',
    'LOFOTEN SJØPRODUKTER A/S',
    'LOVUNDLAKS A/S',
    'MAINSTREAM NORWAY AS',
    'MARØ HAVBRUK A/S',
    'MOWI FEED AS',
    'NORDFJORD OPPDRETT AS',
    'NRS FARMING AS',
    'ØYFISK A/S',
    'ØYLAKS A/S',
    'SALAKS NY AS',
    'SØR FARMING AS',
    'SUNNHORDLAND FJORDBRUK AS',
    'TELAVÅG FISKEOPPDRETT A/S',
    'TROLAND LAKSEOPPDRETT A/S',
    'TROMS FYLKESKOMMUNE',
    'TYSNES FJORDBRUK A/S',
    'VAL SKOLER AS',
    'VAL VGS AS',
    'VEGA SJØFARM AS',
    'WILSGÅRD FISKEOPPDRETT A/S'
]

organizationNames = [ 
    'BLOM FISKEOPPDRETT AS',
    'ARNØY LAKS AS',
    'NORDFJORD OPPDRETT AS',
    'NORDLAKS OPPDRETT AS'
]

X = []
Y = []

allNumMechanicalTreatments = []

counter = 0

newOrganizationNames = []

fmt = '{:<20} {:<20} {:<20} {:<20} {:<20}'

for organizationName in organizationsNameToNo:
#for organizationName in organizationNames:
    if counter > 1000:
        continue

    if organizationsNameToNo[organizationName] is None or organizationName in duplicateOrganizationList:
        continue
   
    if organizationName in organizationLatitudeHistory:
        mechanicalTreatmentWeeks = mechanicalTreatmentWeeksHistory[organizationName]
        #mechanicalBeforeTreatmentMobile = mechanicalBeforeTreatmentMobileHistory[organizationName]
        organizationLatitude = organizationLatitudeHistory[organizationName]
        numMechanicalTreatments = numMechanicalTreatmentsHistory[organizationName]
    else:    
        #(mechanicalTreatmentWeeks, mechanicalBeforeTreatmentMobile, numMechanicalTreatments) = getMobileAnalysis(organizationName, startYear, startWeek, endYear, endWeek)
        (mechanicalTreatmentWeeks, organizationLatitude, numMechanicalTreatments) = getMobileAnalysis(organizationName, startYear, startWeek, endYear, endWeek)

        mechanicalTreatmentWeeksHistory[organizationName] = mechanicalTreatmentWeeks
        #mechanicalBeforeTreatmentMobileHistory[organizationName] = mechanicalBeforeTreatmentMobile
        organizationLatitudeHistory[organizationName] = organizationLatitude
        numMechanicalTreatmentsHistory[organizationName] = numMechanicalTreatments
        
    if mechanicalTreatmentWeeks is None or numMechanicalTreatments < 5:
        continue
    
    newOrganizationNames.append(organizationName)
    
    print('Processing %s' % (organizationName, ))
    
    counter = counter + 1
    
    #X.append(mechanicalBeforeTreatmentMobile)
    X.append(organizationLatitude)
    Y.append(mechanicalTreatmentWeeks * 7)
    allNumMechanicalTreatments.append(numMechanicalTreatments)
    #getReport(organizationName, startYear, startWeek, endYear, endWeek)
    
ordering = np.argsort(Y)[::-1]
    
print(fmt.format('FARM NAME', 'Latitude', '# Treatments', 'Days b/w treatment', 'Percentile'))

for index, order in enumerate(ordering):
    organizationName = newOrganizationNames[order]
    numMechanicalTreatments = allNumMechanicalTreatments[order]
    #mechanicalBeforeTreatmentMobile = X[order]
    organizationLatitude = X[order]
    mechanicalTreatmentWeeks = Y[order]
    
    #output = fmt.format(organizationName[0:15], numMechanicalTreatments, '%0.2f' % (mechanicalBeforeTreatmentMobile, ), int(mechanicalTreatmentWeeks), '%0.1f' % (index / len(ordering) * 100, ))
    output = fmt.format(organizationName[0:15], '%0.2f' % (organizationLatitude, ), numMechanicalTreatments, int(mechanicalTreatmentWeeks), '%0.1f' % (index / len(ordering) * 100, ))
    print(output)
    


In [ ]:
fig, ax = plt.subplots()

fig.set_size_inches(14, 10)

ax.scatter(X, Y)

for i, organizationName in enumerate(newOrganizationNames):
    ax.annotate('%s (%i)' % (organizationName, allNumMechanicalTreatments[i]), (X[i], Y[i]))
    
ax.set_title('Latitude vs Days between treatment')
ax.set_xlabel('Latitude')
ax.set_ylabel('Days between treatment')


In [ ]:
# subset = allMetrics['afterTreatmentAdultFemale'] < 1

# plt.hist(allMetrics['afterTreatmentAdultFemale'][subset], bins=20)

In [ ]:
# subset = allMetrics['afterTreatmentMobile'] < 1

# plt.hist(allMetrics['afterTreatmentMobile'][subset], bins=20)

In [ ]:
# subset = allMetrics['afterTreatmentStationary'] < 1

# plt.hist(allMetrics['afterTreatmentStationary'][subset], bins=20)

In [ ]:
# subset = allMetrics['beforeTreatmentAdultFemale'] < 1

# plt.hist(allMetrics['beforeTreatmentAdultFemale'][subset], bins=20)

In [ ]:
# subset = allMetrics['beforeTreatmentMobile'] < 10

# plt.hist(allMetrics['beforeTreatmentMobile'][subset], bins=20)

In [ ]:
# subset = allMetrics['beforeTreatmentStationary'] < 5

# plt.hist(allMetrics['beforeTreatmentStationary'][subset], bins=20)

In [ ]:
# subset = np.where((treatmentEffectAdultFemale < 0.5) &  (treatmentEffectAdultFemale > -0.25))

# x = treatmentEffectAdultFemale[subset]
# Y = allTreatmentWeeks[subset]

# X = x.reshape(-1, 1)

# plt.plot(X, Y, 'p')

# reg = LinearRegression().fit(X, Y)

# print(reg.intercept_, reg.coef_)
# print(reg.score(X, Y))

# predTreatmentWeeks = reg.predict(X)

# plt.plot(X, predTreatmentWeeks, '-')

In [ ]:
# subset = np.where((afterTreatmentStationary < 0.5))

# x = afterTreatmentStationary[subset]
# Y = allTreatmentWeeks[subset]

# X = x.reshape(-1, 1)

# plt.plot(X, Y, 'p')

# reg = LinearRegression().fit(X, Y)

# print(reg.intercept_, reg.coef_)
# print(reg.score(X, Y))

# predTreatmentWeeks = reg.predict(X)

# plt.plot(X, predTreatmentWeeks, '-')

In [ ]:
# subset = np.where((afterTreatmentMobile < 0.5))

# x = afterTreatmentMobile[subset]
# Y = allTreatmentWeeks[subset]

# X = x.reshape(-1, 1)

# plt.plot(X, Y, 'p')

# reg = LinearRegression().fit(X, Y)

# print(reg.intercept_, reg.coef_)
# print(reg.score(X, Y))

# predTreatmentWeeks = reg.predict(X)

# plt.plot(X, predTreatmentWeeks, '-')

In [ ]:
# subset = np.where((afterTreatmentAdultFemale < 0.5))

# x = afterTreatmentAdultFemale[subset]
# Y = allTreatmentWeeks[subset]

# X = x.reshape(-1, 1)

# plt.plot(X, Y, 'p')

# reg = LinearRegression().fit(X, Y)

# print(reg.intercept_, reg.coef_)
# print(reg.score(X, Y))

# predTreatmentWeeks = reg.predict(X)

# plt.plot(X, predTreatmentWeeks, '-')